# DECORATORS



## Decorators là gì?

là một mẫu thiết kế cấu trúc cho phép lập trình viên mở rộng và chỉnh sửa biểu hiện (behavior) của một hàm, một phương thức hay một lớp mà không cần phải thay đổi bên trong code.

Ý tưởng chính là ta đặt một đối tượng khả gọi (**callable object**), chức năng (**functionality**) mà ta muốn thay đổi *bên trong* của đối tượng với biểu hiện mới. Do đó, decorator chỉ là một wrapper, góp xung quanh đối tượng ban đầu. Trong hầu hết các trường hợp, ta dùng decorator để truyền một hàm như một argument vào decorator để gọi hàm này sau đó, và thể hiện vài hành động trước và sau khi gọi.

## Cú pháp

`@` ngay trước tên của decorator. Decorator được gọi ngay lập tức trước phần thân của hàm.

In [1]:
def our_decorator(other_func):
    def wrapper(args_for_function):
        print('This happens before we call the function')
        return other_func(args_for_function)

    return wrapper

@our_decorator
def greet(name):
    print('Hello,', name)

In [2]:
greet('Susie')

This happens before we call the function
Hello, Susie


## Tại sao phải dùng decorator?

Đơn giản vì nó làm code clean và readable. 

Ví dụ này, ta muốn tính thời gian thực thi của mỗi code block.

In [3]:
import time

def func1(args_for_function):
    start = time.time()  # gets the current time
    ...                  # something happens here
    end = time.time()
    print('func1 takes', end - start, 'seconds')


def func2(args_for_function):
    start = time.time()
    ...
    end = time.time()
    print('func2 takes', end - start, 'seconds')

Trông khá phiền toái mỗi khi muốn tính cho hàng chục code block khác, chưa kể phần code này không liên quan gì đến mục tiêu chính của hàm.

Như vậy, với decorator thì ta sẽ được code clean hơn như này:

In [4]:
def timer(func):
    def wrapper(args_for_function):
        start = time.time()
        func(args_for_function)
        end = time.time()
        print('func takes', end - start, 'seconds')

    return wrapper


@timer
def func1(args_for_function):
    ...  # something happens here

# DECORATORS IN OOP

Thảo luận về 3 built-in decorators chính là: `staticmethod`, `classmethod`, `property`.

## @staticmethod

`@staticmethod` decorator được sử dụng để bind một hàm vào một class như một method của class đó. 

Ví dụ dưới đây, `CharType` class và phương thức `get_type` để lấy type của một ký tự:

In [2]:
class CharType(object):

    @staticmethod
    def get_type(char):
        if char.isalpha():
            return 'letter'
        elif char.isdigit():
            return 'digit'
        else:
            return 'other'


print(CharType.get_type('a'))    # letter: wtihout creating instance
print(CharType().get_type('1'))  # digit: with creating instance

letter
digit


Lưu ý rằng, static method không cần một tham số bắt buộc nào cả, khác với instance method cần phải có `self`, cũng như không cần `cls` giống như class method. Mặc dù một static method thuộc về một lớp và tất cả các instance của lớp đó, nó không thể access được bất kỳ phần bên trong của instance nào. Method này được dùng cho mục đích thuận tiện hoặc thiết kế code tốt.

## @classmethod

Class method chỉ cần access vào những attributes, properties chung của class. Do đó, tham số đầu tiên nó cần luôn là `cls`.

Trong ví dụ dưới đây, `User` và một string chứa thông tin cả `name` và `surname`. Cần phải process string này để nhận 2 attributes vào instance.

In [3]:
class User(object):

    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    def get_info(self):
        return self.name + ' (' + self.surname + ')'

    @classmethod
    def from_string(cls, data):
        name, surname = data.split(' ')
        return cls(name, surname)  # passing the string values to the initialization call


user = User.from_string('Santa Claus')  # using the class name to call the method
print(user.get_info())  # Santa (Claus)

Santa (Claus)


Có thể thấy, class method không cần attribute của một instance cụ thể, thay vào đó cần thông tin của class cho một vài mục đích nào đó. Thông thường là cho mục đích thay thế constructor. Khi đó, nếu thông tin yêu cần cần được lấy từ nguồn bên ngoài, một file chẳng hạn, thì trong trường hợp này, ta không thể pass data trực tiếp vào class instance được. Ta cần dùng một vài kỹ thuật preprocess trước khi thật sự construct instance.

Tuy nhiên, ta cũng có thể gọi `from_string` từ instance, có điều trông không hợp lý lắm.

In [4]:
user2 = user.from_string('Father Christmas')
user2.get_info()  # Father (Christmas)

'Father (Christmas)'

## @staticmethod vs @classmethod

Những điểm khác biệt chính của 2 phương thức này là:

-   Static method không cần `cls` hay `self` như tham số, do đó ta không cần operate điều gì lên class hay instance trong phạm vi phương thức cả. Nhìn chung, static method hoạt động tương tự như một hàm bên ngoài class và thường dùng như một ultility methods.

- Class method luôn cần `cls` là tham số đầu tiên. Thường được dùng như alternative constructor, khi mà có thể tạo nhưng class object cho nhiều trường hợp khác nhau.

## @property

@property dùng để access method như một property. Ví dụ:

In [5]:
class User:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        self.full_name = self.name + ' ' + self.surname


user = User('Santa', 'Claus')
print(user.full_name)  # Santa Claus

Santa Claus


Giờ đổi `name` thành `'Father'` thì `full_name` vẫn y như cữ (vì giá trị đã được lưu từ ban đầu lúc khởi tạo instance). Do đó, ta cần phải dùng `@property` để mỗi khi gọi `full_name` nó sẽ luôn cập nhật từ `name` và `surname`.

In [6]:
class User:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname

    @property
    def full_name(self):
        return self.name + ' ' + self.surname


user = User('Santa', 'Claus')
print(user.full_name)  # Santa Claus

user.name = 'Father'
print(user.name)  # Father
print(user.full_name)  # Father Claus

Santa Claus
Father
Father Claus
